In [ ]:
!pip install -q transformers torch scikit-learn pandas datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import os
import torch
import json
from transformers import Trainer, DistilBertForSequenceClassification, DistilBertTokenizer, TrainingArguments
from sklearn.model_selection import train_test_split
from google.colab import drive

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
drive.mount('/content/drive')
trainFilePath = "/content/drive/My Drive/USF/Fall 2024/Natural Language Processing/hw4/redditComments_train.jsonlist"
testFilePath = "/content/drive/My Drive/USF/Fall 2024/Natural Language Processing/hw4/redditComments_test_notGraded.jsonlist"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Check if GPU is available and set device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA GPU device is available.")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU.")

CUDA GPU device is available.


In [ ]:
trainLabels = []
trainTexts = []

# Load training data
with open(trainFilePath, 'r') as f:
    for line in f:
        entry = json.loads(line)
        trainLabels.append(entry['subreddit'])
        trainTexts.append(entry['body'])

# Split into training and validation sets
trainTexts, valTexts, trainLabels, valLabels = train_test_split(trainTexts, trainLabels, test_size=0.2, random_state=42)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
trainEncodings = tokenizer(trainTexts, truncation=True, padding=True)
valEncodings = tokenizer(valTexts, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
labelToID = {label: idx for idx, label in enumerate(set(trainLabels))}
idToLabel = {idx: label for label, idx in labelToID.items()}

# Convert labels to IDs
trainLabels = [labelToID[label] for label in trainLabels]
valLabels = [labelToID[label] for label in valLabels]

# Check the number of classes
num_classes = len(labelToID)
print("Number of classes:", num_classes)

Number of classes: 4


In [ ]:
class SubredditDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create dataset objects
trainDataset = SubredditDataset(trainEncodings, trainLabels)
valDataset = SubredditDataset(valEncodings, valLabels)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_classes)

training_args = TrainingArguments(
    output_dir="./training_results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir='./training_results/logs',
    fp16=True if torch.cuda.is_available() else False
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainDataset,
    eval_dataset=valDataset
)

# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.608700,0.405466
2,0.316900,0.410907


TrainOutput(global_step=1960, training_loss=0.41898769456513074, metrics={'train_runtime': 439.8133, 'train_samples_per_second': 71.303, 'train_steps_per_second': 4.456, 'total_flos': 4154325789573120.0, 'train_loss': 0.41898769456513074, 'epoch': 2.0})

In [ ]:
from huggingface_hub import login

# Log in to Hugging Face; replace "YOUR_ACCESS_TOKEN" with your actual token
login("YOUR_ACCESS_TOKEN")

In [ ]:
model_save_path = "/content/drive/My Drive/USF/Fall 2024/Natural Language Processing/hw4/trained_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('/content/drive/My Drive/USF/Fall 2024/Natural Language Processing/hw4/trained_model/tokenizer_config.json',
 '/content/drive/My Drive/USF/Fall 2024/Natural Language Processing/hw4/trained_model/special_tokens_map.json',
 '/content/drive/My Drive/USF/Fall 2024/Natural Language Processing/hw4/trained_model/vocab.txt',
 '/content/drive/My Drive/USF/Fall 2024/Natural Language Processing/hw4/trained_model/added_tokens.json')

In [ ]:
# Load the test data
testLabels = []
testTexts = []

with open(testFilePath, 'r') as f:
    for line in f:
        entry = json.loads(line)
        testLabels.append(entry['subreddit'])
        testTexts.append(entry['body'])

# Convert test labels to numeric IDs using the same mapping as training
testLabels = [labelToID.get(label, -1) for label in testLabels]  # Use -1 for unknown labels if any

# Tokenize test data
testEncodings = tokenizer(testTexts, truncation=True, padding="longest", return_tensors="pt")

In [ ]:
# Define a custom dataset for the test set
class SubredditTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the test dataset
testDataset = SubredditTestDataset(testEncodings, testLabels)

In [ ]:
# Evaluate the model on the test dataset
eval_results = trainer.evaluate(eval_dataset=testDataset)
print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 0.3132633566856384, 'eval_runtime': 0.5811, 'eval_samples_per_second': 344.153, 'eval_steps_per_second': 43.019, 'epoch': 2.0}


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Put model in evaluation mode and move to the appropriate device
model.eval()
model.to(device)

# Generate predictions on the test set
predictions = []
with torch.no_grad():
    for i in range(len(testDataset)):
        input_ids = testDataset[i]['input_ids'].unsqueeze(0).to(device)
        attention_mask = testDataset[i]['attention_mask'].unsqueeze(0).to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        pred = torch.argmax(outputs.logits, dim=1).item()
        predictions.append(pred)

# Calculate accuracy and other metrics
accuracy = accuracy_score(testLabels, predictions)
print(f"Accuracy on test set: {accuracy:.4f}")
print("\nClassification Report:\n", classification_report(testLabels, predictions, target_names=idToLabel.values()))

Accuracy on test set: 0.8900

Classification Report:
               precision    recall  f1-score   support

        Nerf       0.89      0.84      0.87        50
newToTheNavy       0.91      0.96      0.93        50
     ukulele       0.90      0.88      0.89        50
  MarioMaker       0.86      0.88      0.87        50

    accuracy                           0.89       200
   macro avg       0.89      0.89      0.89       200
weighted avg       0.89      0.89      0.89       200

